In [24]:
%config IPCompleter.greedy=True
import os.path
import csv
import psycopg2
import petl as etl
from sqlalchemy import create_engine
from collections import OrderedDict
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, BlobBlock

In [2]:
account_name='datalake2dictas'
account_key='wlxJHcWvtVhPpL/zs6l+F1bJGKZnJ4HppZcVyh+ns32oH46E3dY/HBLau3V6um9hv+KZf/3mXEAL5nHD41X3jg=='
container_from = 'transient'
container_to = 'raw'
upload_file = '/opt/jupyter_etl_dictas/datalake/transient/beneficiario/beneficiarios_atualizado.csv'
_LOCALDIR = '/opt/jupyter_etl_dictas/datalake/transient'

In [19]:
cc_transient = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_from, credential=account_key)
cc_raw = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_to, credential=account_key)
cc_refined = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name='refined', credential=account_key)

In [4]:
_TEMP_FILE='datalake/temp'
_METADADOS='metadado/list.csv'
_SOURCE='17'
_PROC_FILES='datalake/proc'

#### Lista Blobs

In [5]:
# List the blobs in the container.
print("\nList blobs in the container")
blob_list = cc_raw.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')


List blobs in the container
BENEFICIARIO.csv

CONSULTAS.csv

EMPRESA.csv

PRESTADORES.csv

RECEITAS.csv

SERVICOS.csv

VIDAS.csv



#### Busca lista de arquivos

In [6]:
with open(_METADADOS, newline='') as lista:
    reader = csv.DictReader(lista)
    for nm_file in reader:
        print(nm_file['ARQUIVOS'])

BENEFICIARIO.csv
PRESTADORES.csv
EMPRESA.csv
SERVICOS.csv
VIDAS.csv
CUSTOS.csv
CONSULTAS.csv
DIARIAS.csv
RECEITAS.csv


#### Teste de carga blog

In [9]:
try:
    with open(_METADADOS, newline='') as lista:
        reader = csv.DictReader(lista)
        for nm_file in reader:
            file = nm_file['ARQUIVOS']
            blob = cc_raw.get_blob_client(file)
            upload_file = f'{_TEMP_FILE}/{_SOURCE}/{file}'
            t_arq = os.path.isfile(upload_file)
            if t_arq == True:
                 with open (upload_file, "rb") as data :
                    blob.upload_blob(data, overwrite = True)
                    print("Arquivo "+ file +" carregado")
            else:
                print("Arquivo "+ file +" não encontrado")
        
finally:
    print("Tudo carregado")

Arquivo BENEFICIARIO.csv não encontrado
Arquivo PRESTADORES.csv não encontrado
Arquivo EMPRESA.csv não encontrado
Arquivo SERVICOS.csv não encontrado
Arquivo VIDAS.csv carregado
Arquivo CUSTOS.csv não encontrado
Arquivo CONSULTAS.csv não encontrado
Arquivo DIARIAS.csv não encontrado
Arquivo RECEITAS.csv não encontrado
Tudo carregado


### Carrega blob Beneficiario

#### Verifica Arquivo no Container

In [7]:
blob_list = cc_raw.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    

BENEFICIARIO.csv

CONSULTAS.csv

EMPRESA.csv

PRESTADORES.csv

RECEITAS.csv

SERVICOS.csv

VIDAS.csv



#### Faz Download para diretório local

In [13]:
#arquivo=BENEFICIARIO.csv
#blob_list = cc_transient.list_blobs()
#for blob in blob_list:
    #print(blob.name + '\n')l
    #nm_arq=blob.name
nm_arq="BENEFICIARIO.csv"
with open (f'{_PROC_FILES}/{_SOURCE}/{nm_arq}', 'wb') as data_from :
    data_from.write(cc_raw.get_blob_client(nm_arq).download_blob ().readall ())

#### Carrega dados para transformação

In [15]:
table = etl.fromcsv(f'{_PROC_FILES}/{_SOURCE}/{nm_arq}', delimiter='|')
table

pk_beneficiario,nr_beneficiario,nr_beneficiario_tit,fk_empresa,nome,cpf,dt_nascimento,cod_plano,descricao_plano,copart_percentual,limite_copart,tipo_acomodacao,abrangencia_plano,grau_dependencia,dt_inclusao,dt_exclusao,nome_mae,sexo,tipo_contrato,endereco,bairro,cidade,uf,cep,tipo_cliente,nr_cartaonacionalsaude,plano_regulamentado,descricao_entidade,tipo_pessoa,acao_judicial,fk_operadora
00000005001570,00010001001570032,00010001001570008,17155730000164,MARCO AURELIO GUIMARAES MONTEIRO,48125857672,14/03/1961,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,CONJUGE / COMPANHEIRO,08/01/2007,23/02/2011,FELISBINA GUIMARAES MONTEIRO,M,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,706201064694669,SIM,CEMIG GT,FISICA,2,17
00000005012837,00010001012837000,00010001012837000,17155730000164,MARCO AURELIO GUIMARAES MONTEIRO,48125857672,14/03/1961,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,TITULAR,23/02/2011,09/07/2015,FELISBINA GUIMARAES MONTEIRO,M,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,706201064694669,SIM,CEMIG HOLGING,FISICA,2,17
00000006000003,00010001000003010,00010001000003002,17155730000164,MARIA EDVIGES CABRAL DE AZEVEDO,37025546615,14/08/1956,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,CONJUGE / COMPANHEIRO,10/07/2015,,ANNA ANTONIA CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,700503121519158,SIM,CEMIG D,FISICA,2,17
00000006001570,00010001001570008,00010001001570008,17155730000164,MARIA EDVIGES CABRAL DE AZEVEDO,37025546615,14/08/1956,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,TITULAR,02/01/2005,23/02/2011,ANNA ANTONIA CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190081,GRUPO CEMIG,700503121519158,SIM,CEMIG GT,FISICA,2,17
00000007012837,00010001012837019,00010001012837000,17155730000164,MARINA AZEVEDO MONTEIRO,12688868667,09/11/1993,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,FILHO / FILHA,23/02/2011,09/07/2015,MARIA EDVIGES CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,703401695595500,SIM,CEMIG HOLGING,FISICA,2,17


In [26]:
table1 = etl.convert(table, {'pk_beneficiario': str,
                             'fk_operadora': int
                            })
table1

pk_beneficiario,nr_beneficiario,nr_beneficiario_tit,fk_empresa,nome,cpf,dt_nascimento,cod_plano,descricao_plano,copart_percentual,limite_copart,tipo_acomodacao,abrangencia_plano,grau_dependencia,dt_inclusao,dt_exclusao,nome_mae,sexo,tipo_contrato,endereco,bairro,cidade,uf,cep,tipo_cliente,nr_cartaonacionalsaude,plano_regulamentado,descricao_entidade,tipo_pessoa,acao_judicial,fk_operadora
00000005001570,00010001001570032,00010001001570008,17155730000164,MARCO AURELIO GUIMARAES MONTEIRO,48125857672,14/03/1961,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,CONJUGE / COMPANHEIRO,08/01/2007,23/02/2011,FELISBINA GUIMARAES MONTEIRO,M,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,706201064694669,SIM,CEMIG GT,FISICA,2,17
00000005012837,00010001012837000,00010001012837000,17155730000164,MARCO AURELIO GUIMARAES MONTEIRO,48125857672,14/03/1961,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,TITULAR,23/02/2011,09/07/2015,FELISBINA GUIMARAES MONTEIRO,M,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,706201064694669,SIM,CEMIG HOLGING,FISICA,2,17
00000006000003,00010001000003010,00010001000003002,17155730000164,MARIA EDVIGES CABRAL DE AZEVEDO,37025546615,14/08/1956,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,CONJUGE / COMPANHEIRO,10/07/2015,,ANNA ANTONIA CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,700503121519158,SIM,CEMIG D,FISICA,2,17
00000006001570,00010001001570008,00010001001570008,17155730000164,MARIA EDVIGES CABRAL DE AZEVEDO,37025546615,14/08/1956,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,TITULAR,02/01/2005,23/02/2011,ANNA ANTONIA CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190081,GRUPO CEMIG,700503121519158,SIM,CEMIG GT,FISICA,2,17
00000007012837,00010001012837019,00010001012837000,17155730000164,MARINA AZEVEDO MONTEIRO,12688868667,09/11/1993,0001,PROSAUDE INTEGRADO,,,APARTAMENTO STANDART,NACIONAL,FILHO / FILHA,23/02/2011,09/07/2015,MARIA EDVIGES CABRAL DE AZEVEDO,F,COLETIVO EMPRESARIAL,RUA MATO GROSSO,SANTO AGOSTINHO,BELO HORIZONTE,MG,30190088,GRUPO CEMIG,703401695595500,SIM,CEMIG HOLGING,FISICA,2,17


#### Add fk_empresa

In [17]:
#table2 = etl.addfields(table1,[('cd_empresa',_SOURCE)])
etl.tocsv(table1, f'{_PROC_FILES}/{_SOURCE}/t{nm_arq}', delimiter='|')

In [20]:
upload_file=f'{_PROC_FILES}/{_SOURCE}/t{nm_arq}'
#print(upload_file)
try:
    with open (upload_file, "rb") as data :
        #print(data)
        cc_refined.upload_blob(nm_arq, data, overwrite = True)
finally:
    print("Tudo carregado")

Tudo carregado


In [28]:
engine = create_engine(
    "postgresql+psycopg2://giovani.stefani@prd-dictas-postgresql-server.postgres.database.azure.com:B7hqx9mFi19J8BTE@prd-dictas-postgresql-server.postgres.database.azure.com:5432/dictas")

In [30]:
etl.todb(table1, engine, 'beneficiario')

DataError: (psycopg2.errors.DatetimeFieldOverflow) date/time field value out of range: "14/03/1961"
LINE 1: ...MARCO AURELIO GUIMARAES MONTEIRO', '48125857672', '14/03/196...
                                                             ^
HINT:  Perhaps you need a different "datestyle" setting.

[SQL: INSERT INTO "beneficiario" ("pk_beneficiario", "nr_beneficiario", "nr_beneficiario_tit", "fk_empresa", "nome", "cpf", "dt_nascimento", "cod_plano", "descricao_plano", "copart_percentual", "limite_copart", "tipo_acomodacao", "abrangencia_plano", "grau_dependencia", "dt_inclusao", "dt_exclusao", "nome_mae", "sexo", "tipo_contrato", "endereco", "bairro", "cidade", "uf", "cep", "tipo_cliente", "nr_cartaonacionalsaude", "plano_regulamentado", "descricao_entidade", "tipo_pessoa", "acao_judicial", "fk_operadora") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)]
[parameters: ('00000005001570', '00010001001570032', 